In [1]:
from collections import namedtuple
from gensim.models import doc2vec
from konlpy.tag import Twitter
import multiprocessing
from pprint import pprint
from gensim.models import Doc2Vec
from sklearn.linear_model import LogisticRegression
import numpy
import pickle

twitter = Twitter()
def read_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
    return data

def tokenize(doc):
# norm, stem은 optional
    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

c:\users\yooso\anaconda\envs\konlpy\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\users\yooso\anaconda\envs\konlpy\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [2]:

# load train data
doc_vectorizer = Doc2Vec.load('model_jh/리뷰모음_train_랄라.model')

# doc2vec 에서 필요한 데이터 형식으로 변경
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

# load the model from disk
filename = 'model_jh/finalized_model_gbn.sav'
# 실제 분류 확인
gnb = pickle.load(open(filename, 'rb'))

def para2st(para):
    re = []
    para = para.replace("했다","했다.").replace("했다..","했다.").replace("했다.","했다.\n").replace("니다","니다.").replace("니다..","니다.").replace("니다.","니다.\n").replace("까?","까?\n").replace("다!","다!\n").replace("요.","요.\n").replace("요!","요!\n")
    para = para.split('\n')
    split_st = [item.strip() for item in para]
    run_docs = [(tokenize(item),-1) for item in split_st]
    
    tagged_run_docs = [TaggedDocument(d, [c]) for d, c in run_docs]
    
    # 분류를 위한 피쳐 생성
    run_x = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_run_docs]

    pred_y = gnb.predict(run_x)
    return split_st, pred_y


In [ ]:
import glob
import os

from pandas import DataFrame, read_csv
import pandas as pd #this is how I usually import pandas
import matplotlib #only needed to determine Matplotlib version number

name_set = []
sum_avg_set = []
sum_avg=0.0
folderlist = glob.glob('./data/*')
for fol in folderlist:
    if os.path.isdir(fol):
        courses = []
        comments_nums = []
        hits_nums = [] 
        print('folder:',fol)
        filelist = glob.glob(fol+'/*.txt')
        folder = fol[7:]
        print('folder:',folder)
        for file in filelist:
            #print(file)
            
            input_file = file.replace('\\','/')
            course = input_file.split('/')[3][:-9]
            print(input_file,end='\t')
            #2016_수능특강_국어_화작문_orbi
            
            run_data = read_data(input_file)
            
            #결과
            result_set = []
            for row in run_data[1:]:
                if len(row) >= 3:
                    result_set.append(para2st(row[2]))
            sum_avg = 0.0
            sum_ =0 
            for item in result_set:
                #for i in range(len(item[0])):
                    #print(item[1][i],item[0][i], end='\n')
                sum_ = sum(item[1].astype(numpy.int32))/len(item[1])
                #print(sum(item[1].astype(numpy.int32))/len(item[1]))
            try:
                sum_avg = sum_/len(result_set)
                print(sum_avg)
                name_set.append(course)
                sum_avg_set.append(sum_avg)
            except:
                sum_avg=0
        BabyDataSet = list(zip(name_set,sum_avg_set))
        df = pd.DataFrame(data = BabyDataSet, columns=['Names', 'sentiment_score'])
        output_file = '긍부정_'+folder
        print('output_file_name',output_file)
        df.to_csv(output_file+'.csv', encoding='utf-8-sig',index=False,header=True)
        #긍부정_2016_수능특강_국어_화작문_orbi
        
        
    #df=pd.DataFrame()
    #df.to_csv(output_file+'_실험.csv', encoding='utf-8-sig', index=False)

c:\users\yooso\anaconda\envs\konlpy\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\yooso\anaconda\envs\konlpy\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


folder: ./data\2014
folder: 2014
./data/2014/지식인_수특 경제 2014.txt	0.00018593408636638313
./data/2014/지식인_수특 국어-A형 2014.txt	0.00013061650992685477
./data/2014/지식인_수특 국어-B형 2014.txt	7.396449704142012e-05
./data/2014/지식인_수특 기하와벡터 2014.txt	9.46969696969697e-05
./data/2014/지식인_수특 동아시아사 2014.txt	0.00011042402826855124
./data/2014/지식인_수특 물리1 2014.txt	0.0
./data/2014/지식인_수특 물리2 2014.txt	0.0
./data/2014/지식인_수특 미적분과통계기본 2014.txt	6.482982171799028e-05
./data/2014/지식인_수특 법과정치 2014.txt	0.00010049241282283187
./data/2014/지식인_수특 사회문화 2014.txt	4.317789291882556e-05
./data/2014/지식인_수특 생물1 2014.txt	3.132832080200501e-05
./data/2014/지식인_수특 생물2 2014.txt	4.100041000410004e-05
./data/2014/지식인_수특 생활과윤리 2014.txt	2.774694783573807e-05
./data/2014/지식인_수특 세계지리 2014.txt	